## Imports/Setup

In [5]:
%run ../00_functions/00_custom_functions.ipynb
standard_setup(extra_packages=True, verbose=False)
data_ingestion_prep(start_yearmonth='1979-01', end_yearmonth='2022-12', verbose=False)

env: ESMFMKFILE=/home/ds4114/.conda/envs/leap_test202308-3/lib/esmf.mk
Files will be outputed as: .nc
ECMWF CDS API key already installed


## Collection: Wind Speed

#### ECMWF ERA5 Wind

In order to get the wind standard deviation, we need to download hourly data so it is very large. I have only done shown one example here because we already have the data downloaded/processed.

In [19]:
variable_list = ['10m_u_component_of_wind', '10m_v_component_of_wind']
wind_destination_folder = global_vars['download_folder']+r'Wind/originals/'
for year in range(2000, 2000+1, 1):  #just one year as an example
    wind_destination_filename = ''
    months = []
    if year == datetime.datetime.now().year:   #if a partial year; this prevents erorrs trying to use the API to get future/non-existant data    
        months = [i for i in range(1,datetime.datetime.now().month -1)]  
        wind_destination_filename = 'Wind_ECMWF_ERA5-monthly-reanalysis-10m-uv-wind_'+str(year)+'.nc'  
        #Note, partial years will not automatically be overwritten so you must manually clean up unneeded files
    else:
        wind_destination_filename = 'Wind_ECMWF_ERA5-monthly-reanalysis-10m-uv-wind_'+str(year)+'.nc'
        months = [i for i in range(1,12+1)]

    cdsapi_custom_download(year, months, variable_list, wind_destination_folder, wind_destination_filename, overwrite=False, create_dest=False)

print('Wind Complete')

2023-12-17 15:21:12,017 INFO Welcome to the CDS
2023-12-17 15:21:12,018 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
2023-12-17 15:21:12,145 INFO Request is queued
2023-12-17 15:21:13,249 INFO Request is running
2023-12-17 15:21:20,687 INFO Request is completed
2023-12-17 15:21:20,689 INFO Downloading https://download-0009-clone.copernicus-climate.eu/cache-compute-0009/cache/data0/adaptor.mars.internal-1702844476.0123901-17563-1-d985df94-8350-41b8-b83e-99a57ae019e6.nc to ../tmp/Wind_ECMWF_ERA5-monthly-reanalysis-10m-uv-wind_2000.nc (47.5M)
2023-12-17 15:21:23,501 INFO Download rate 16.9M/s  


Transferring to Destination /data/artemis/workspace/ds4114/online_data/Wind/originals/Wind_ECMWF_ERA5-monthly-reanalysis-10m-uv-wind_2000.nc...
Complete


In [2]:
#example from before: xr.open_dataset('/data/artemis/observations/ERA5/Winds_2020-01.nc')

## Processing: Wind Speeed

#### ECMWF ERA5 Wind

To get aggregate speed and wind std. Again, this is just an example because we dont have all the data downloaded

In [7]:
wind = xr_open_mfdataset_custom(os.path.join(global_vars['download_folder'],'Wind/originals/','Wind_ECMWF_ERA5-monthly-reanalysis-10m-uv-wind_*.nc'))

In [ ]:
wind_cut = wind.sel(time=slice(str(processed_start_yearmonth_back_in_time),str(processed_end_yearmonth))) #already monthly so select desired years

#regrid to flip longitude
wind_regridder = xesmf.Regridder(wind_cut, ideal_grid_back_in_time, 'bilinear', periodic=True) 
wind_cut_out = wind_regridder(wind_cut, keep_attrs=True)
wind_cut_out = wind_cut_out.rename({'latitude': 'ylat','longitude': 'xlon'}) 
wind_cut_out = wind_cut_out.assign_coords(time=ttime_back_in_time)  #will break if we dont have all months downloaded

#mabye combine u and v components here
ws10 = (wind_cut_out.u10**2+wind_cut_out.v10**2)**0.5
wind_out = ws10.to_dataset(name='ws10')

In [ ]:
#merge wind_out and wind_std_out - TBD
#output after
    #output_xarray_with_date(wind_out, os.path.join(data_folder_root,'Wind/processed/'), 'Wind_ECMWF_ERA5-monthly-10m-uv-wind-1x1', filetype=output_file_type, overwrite=False)